In [22]:
from heapq import heappush, heappop
import pandas as pd
import re
import folium
import geohash
import random

In [23]:
# Required Files, those are not public files
# geohashmap = pd.read_csv('*') # 2843 rows × 1714 columns
# speedmap = pd.read_csv('*')
# ttime_lambda = pd.read_csv('*')

In [24]:
ttime_lambda = ttime_lambda.to_dict()
time_lambda = {}
for i in range(len(ttime_lambda['column'])):
    minuate = int(ttime_lambda['column'][i])
    datae = re.findall(r"'(.*?)'", ttime_lambda['tuple'][i])
    if minuate not in time_lambda:
        time_lambda[minuate] = [datae]
    else: time_lambda[minuate].append(datae)
        
# decoding function
base = {}
for i in range(32):
    if i < 10:
        base[str(i)] = '0' * (5-len(bin(i)[2:])) + bin(i)[2:]
    elif i < 17:
        base[chr(i+88)] = '0' * (5-len(bin(i)[2:])) + bin(i)[2:]
    elif i < 19:
        base[chr(i+89)] = '0' * (5-len(bin(i)[2:])) + bin(i)[2:]
    elif i < 21:
        base[chr(i+90)] = '0' * (5-len(bin(i)[2:])) + bin(i)[2:]
    else:
        base[chr(i+91)] = '0' * (5-len(bin(i)[2:])) + bin(i)[2:]

def hashtoarrayindex(hash):
    bi = ''
    for c in hash:
        bi += base[c]
    odd = '0b' # row
    even = '0b' # column
    for i in range(len(bi)//2):
        odd += bi[2*i+1]
        even += bi[2*i]
    return (int(odd, 2)-733067, int(even, 2)-900017)

[['wy7uzqyu', 'wykjhrts'],
 ['wy7vpc8g', 'wy7uzw37'],
 ['wy7uzqy9', 'wy7uznwc'],
 ['wy7uzzxr', 'wy7vpb9c'],
 ['wykh9xmh', 'wykjhrts'],
 ['wy7uy6v8', 'wy7vnbwu'],
 ['wy7vpce1', 'wy7vpc6s'],
 ['wy7vpc8g', 'wy7vp97w'],
 ['wykh6q4s', 'wy7vpbem'],
 ['wykhdqyp', 'wy7uzft7'],
 ['wykh6pe6', 'wy7vp8pm'],
 ['wykh6nqx', 'wy7vpugm'],
 ['wykh6mme', 'wy7vpduz'],
 ['wykhbpu1', 'wy7vpdrm'],
 ['wykhbp70', 'wykj0x2m'],
 ['wy7uzxtn', 'wy7vptqs'],
 ['wykhdvbx', 'wy7vpugq']]

In [26]:
# entity
class Customer(object):
    def __init__(self, index, time, up_pos, down_pos):
        self.index = index
        self.time = time
        self.up_pos = up_pos
        self.down_pos = down_pos

class Car(object):
    def __init__(self, index, capacity, reserved, now_pos, path):
        self.index = index
        self.capacity = capacity # [customer 객체]
        self.reserved = reserved # [customer 객체]
        self.now_pos = now_pos
        self.path = path # [(목적지, 승객을 태울건지)]
        
car_max_cap = 20

In [44]:
import requests

def cal_path(start, end):
    url = "https://apis-navi.kakaomobility.com/v1/directions"
    headers = {
        # "Authorization": "kakao api key"
    }
    params = {
        "origin": f"{start[1]},{start[0]}",
        "destination": f"{end[1]},{end[0]}",
        "waypoints": "",
        "road_details": "True"
    }

    response = requests.get(url, headers=headers, params=params)
    response = response.json()
    print("cal_path", response['routes'][0]['result_msg'])
    
    all_vertices = []
    for route in response['routes']:
        for section in route['sections']:
            for road in section['roads']:
                all_vertices.extend(road['vertexes'])
    vertices_tuples = [(all_vertices[i+1], all_vertices[i]) for i in range(0, len(all_vertices), 2)][::2]
    return vertices_tuples

def cal_speed_path(start_pos, end_pos): # 시작 지점과 끝 지점을 제외한 경로를 만들어주는 함수
    path = []
    freq = 20
    sx, sy = start_pos
    ex, ey = end_pos
    flipx, flipy = 1, 1
    if abs(ey-sy) >= abs(ex-sx):
        period = abs(ey-sy)//freq
        if ex < sx: flipx = -1
        if ey < sy: flipy = -1
        for i in range(period):
            for j in range(abs(sx-ex)):
                temp = (sx + j*flipx, sy + (i+1)*freq*flipy)
                temp_hash = geohashmap.iloc[temp]
                if temp_hash != str(0):
                    path.append(temp)
                    break
    else:
        period = abs(ex-sx)//freq
        if ex < sx: flipx = -1
        if ey < sy: flipy = -1
        for i in range(period):
            for j in range(abs(sy-ey)):
                temp = (sx + (i+1)*freq*flipx, sy + j * flipy)
                temp_hash = geohashmap.iloc[temp]
                if temp_hash != str(0):
                    path.append(temp)
                    break
    return path

def cal_time(start, end):
    dx = abs(start[0] - end[0]) 
    dy = abs(start[1] - end[1])
    # 30.944622355745157 dy 19.047461054233576 dx
    distance = dx * 19 + dy * 31 # 포항시 2843 rows × 1714 columns 기준
    vx, vy = speedmap.iloc[start], speedmap.iloc[end] # km/h
    if vx == 0: vx = 48.95 # 혹여나 데이터가 없는 경우 전체 평균 속력으로 사용 이런 경우 별로 없을듯
    if vy == 0: vy = 48.95
    vmean = (vx + vy)/2
    t = distance / (vmean * 1000/3600) # m/s로 바꿈
    if t <= 60:
        return 60
    return int((t//60) * 60)

def cal_total_time(start, end):
    start = hashtoarrayindex(start)
    end = hashtoarrayindex(end)
    temp_path = cal_speed_path(start,end)
    temp_path = [start] + temp_path + [end]
    total_time = 0
    for i in range(len(temp_path)-1):
        total_time += cal_time(temp_path[i], temp_path[i+1])
    return total_time

def cal_distance(hash1, hash2):
    x1, y1 = hashtoarrayindex(hash1)
    x2, y2 = hashtoarrayindex(hash2)
    return abs(x1-x2) * 19 + abs(y1-y2) * 31

In [222]:
def check_second_element(lst, target):
    for item in lst:
        if item[1] == target:
            return True
    return False

def algorithm(station_pos, station_inf, station, cars): # path를 직접 바꾸지 않도록
    des = [] # 차량의 목적지를 저장, 차량 당 한 곳만 지정
    for i in range(car_number): # 차량들의 목적지를 저장
        now_index = station_pos.index(cars[i].now_pos)
        if i % 2 == 0:
            new_index = (now_index + 1) % len(station_pos)
            if station_inf[new_index] == False: # 예비정류장인 경우 조건 생각
                if not check_second_element(cars[i].capacity, new_index) and sum([len(ele) for ele in station]) != 0 and len(station[new_index])/sum([len(ele) for ele in station]) < 0.1:
                    new_index += 1 # 10% 보다 낮을경우 생략
        else:
            new_index = (now_index - 1) % len(station_pos)
            if station_inf[new_index] == False:
                if not check_second_element(cars[i].capacity, new_index) and sum([len(ele) for ele in station]) != 0 and len(station[new_index])/sum([len(ele) for ele in station]) < 0.1:
                    new_index -= 1
        new_index = new_index % len(station_pos)
        des.append(station_pos[new_index])
        available_num = car_max_cap - len(cars[i].capacity) - len(cars[i].reserved)
        remove_index = []
        for j in range(min(len(station[now_index]), available_num)):
            forward_len = (station[now_index][j][1] - now_index) % len(station_pos)
            backward_len = (now_index - station[now_index][j][1]) % len(station_pos)
            checkoff = False
            if (forward_len <= backward_len) and (i % 2 == 0): checkoff = True
            if (backward_len <= forward_len) and (i % 2 == 1): checkoff = True
            if checkoff and available_num > 0:
                cars[i].reserved.append(station[now_index][j])
                remove_index.append(j)
        print(i,"th car", "go to ride new customer", cars[i].reserved)
        station[now_index] = [station[now_index][k] for k in range(len(station[now_index])) if k not in remove_index] # 예약된 승객은 기다리는게 아니므로 제거
    
    return des

In [223]:
car_number = 4
station = ['wykh8hcp','wy7uzegr', 'wy7uzufk','wy7vpfsc', 'wy7uyxm0','wy7uygct','wy7uzkb4', 'wy7uxsv8']
station_inf = [True, False, True, True, True, True, False, True]
,
class Simulator(object):
    def __init__(self): 
        self.event_list = []
        self.car_number = car_number
        self.initial_pos = ['wy7vpfsc','wy7vpfsc','wy7uygct', 'wy7uygct']
        self.car = [Car(index, [], [], self.initial_pos[index], []) for index in range(self.car_number)] # 차량 번호, 차량 용량, 차량 위치, 차량 목적지 리스트
        self.customer = [] # self.wt length, 맨 뒤에 최신 승객, [2분전, 1분전, 현재 승객], 여기는 geohash로 
        station_num = len(station)
        self.station = [[] for _ in range(station_num)] # 각 정류장 번호 안 원소는 [승객 객체, 내리려는 정류장 번호]로 되어 있음
        self.station_pos = [hashtoarrayindex(ele) for ele in station]
        self.time = 0
        self.trigger = {0: self.car_decision} # "demand" == 0
        for j in range(self.car_number):
            self.trigger[self.en(1, j)] = self.car_updown  # "stop_demand": 1
            self.trigger[self.en(2, j)] = self.car_moving  # "updown": 2
            self.trigger[self.en(3, j)] = self.car_moving  # "stop_middle": 3
        self.wt = 3 # 호출하고 예약이 안될때 최대로 기다리는 분
        self.n = 0 # 운행횟수 저장용
    
    def en(self, event_index, car_index): # event_number
        return (event_index - 1) * self.car_number + car_index + 1
    
    def ci(self, event_number):
        return (event_number - 1)% self.car_number
    
    def nearest_station(self, pos):
        dis_list = [abs(pos[0] - station[0]) * 19 + abs(pos[1] - station[1]) * 31 for station in self.station_pos]
        min_dis = min(dis_list)
        random.seed(4)
        random_value = random.gauss(1520, 520)
        if min_dis > random_value: # 가장 가까운 정류장도 매우 멀면 이용하지 않음, 거리 기반 확률, 설문조사 기반
            return -1
        return dis_list.index(min_dis)
        
    def gen_customer(self):
        if len(self.customer) >= self.wt: self.customer.pop(0)
        self.customer.append([])
        for i, ele in enumerate(time_lambda[self.time//60]): # every_minuate
            temp_cus = Customer(i, self.time, ele[0], ele[1])
            self.customer[-1].append(temp_cus)
            start_station = self.nearest_station(hashtoarrayindex(ele[0])) 
            end_station = self.nearest_station(hashtoarrayindex(ele[1]))
            if start_station > -1 and end_station > -1 and start_station != end_station:
                self.station[start_station] += [(temp_cus, end_station)] # 현재는 기다리다 정류장을 떠나지 않도록 되어 있음
        print("custmoer generation", self.time, len(self.customer[-1]), self.customer[-1][0].time)
    
    def demand(self, max_time, dec_interval=60): # every_minuate
        for i in range(max_time):
            if i % dec_interval == 0: heappush(self.event_list, (i+0.1, 0)) # 결정은 가장 나중에
            if i % 60 == 0: heappush(self.event_list, (i-0.1, -1)) # 손님 반영은 가장 먼저
    
    def car_decision(self):
        paths = algorithm(station, station_inf, self.station, self.car)
        for i in range(car_number):
            store_des[i].append(paths[i])
        print("now_pos", self.time, [station.index(self.car[i].now_pos) for i in range(self.car_number)])
        print("test", [self.car[i].capacity for i in range(self.car_number)])
        print("station", [len(ele) for ele in self.station])
        for i in range(self.car_number):
            self.car[i].path = paths[i]
            store_capacity[i][self.time//60] = self.car[i].capacity
            print(i,"th car", "path", station.index(self.car[i].path))
        print("decision", self.time, "\n")
            
    def car_updown(self, index): 
        now_pos = self.car[index].now_pos
        cap = self.car[index].capacity
        reserved = self.car[index].reserved
        num = [0, 0] # 타거나 내리는 승객의 수
        cus_inf = [] # 승객 정보 외부 변수에 저장용
        
        remove_index = [] # 승객을 내리게 하는 부분
        for i in range(len(cap)):
            if cap[i][1] == station.index(now_pos):
                print(index,"th car", "I let down him")
                cus_inf.append([cap[i][0], self.time])
                remove_index.append(i)
                num[1] += 1
                self.n += 1 # 운행이 완료된 횟수
        self.car[index].capacity = [cap[i] for i in range(len(cap)) if i not in remove_index] # 차량 용량 삭제
        
        remove_index = [] # 승객을 태우게 하는 부분
        for i, cus in enumerate(reserved):
            print(index,"th car", "I ride him")
            cus_inf.append([cus[0], self.time])
            remove_index.append(i)
            self.car[index].capacity.append(cus)
            num[0] += 1
        self.car[index].reserved = [reserved[i] for i in range(len(reserved)) if i not in remove_index]   
        
        print(index,"th car", "updown", index, now_pos, sum(num))
        if sum(num) > 0:
            for ele in cus_inf:
                if ele[0] not in store_cus[index]: store_cus[index][ele[0]] = [ele[1]]
                else: store_cus[index][ele[0]].append(ele[1])
            heappush(self.event_list, (self.time + 60, self.en(2, index))) # 승객을 태웠다면
            
    def car_moving(self, index):
        car = self.car[index]
        print(index,"th car", "moving", station.index(car.now_pos))
        left_time = cal_total_time(car.now_pos, car.path)
        if self.time > 1:
            temp_path = cal_path(geohash.decode(car.now_pos), geohash.decode(car.path))
            store_path[index] += [[pos, False] for pos in temp_path[1:-1]]
            store_path[index].append([geohash.decode(car.path), True])
            car.now_pos = car.path
        print(index,"th car", "moving", station.index(car.now_pos), left_time)
        dest_num = sum(1 for item in car.capacity if item[1] == station.index(car.now_pos))
        if len(car.reserved) > 0 or dest_num > 0:
            heappush(self.event_list, (self.time + left_time, self.en(1, index)))
        else:
            heappush(self.event_list, (self.time + left_time, self.en(2, index)))
            
    def run(self, max_time): # 모든 시간은 초 단위로 사용하자 나중에 확장
        for _ in range(10000):
            if len(self.event_list) == 0 or (max_time < int(abs(round(self.event_list[0][0], 0)))): break # 최대 시간 설정
            t, event = heappop(self.event_list)
            self.time = int(abs(round(t, 0))) 
            if event == -1: self.gen_customer()
            elif event == 0: self.trigger[event]()
            else: self.trigger[event](self.ci(event)) # car_index: (event - 1)% self.car_number 
                        
    def run_step(self, action, time): # 강화학습용
        self.action = action 
        self.run(time)
        cars_now_pos = [self.car[i].now_pos for i in range(self.car_number)]
        cars_now_cus = [self.car[i].capacity for i in range(self.car_number)]
        # return self.customer, cars_now_pos, cars_now_cus # 승객의 전체 위치, 현재 위치, 타고 있는 승객 정보
        
    def reset(self): # 승객이 발생하도록 이벤트를 넣어준다
        self.event_list = []
        self.car = [Car(index, [], [], self.initial_pos[index], []) for index in range(self.car_number)]
        self.customer = []
        self.demand(3600) # 1을 넣으면 1초 단위로 결정, 안 넣으면 1분 단위(승객이 생길 때)로 결정
        for index in range(self.car_number):
            heappush(self.event_list, (1, self.en(2, index))) # 초기 움직임 이니시에이팅
            store_path[index].append([geohash.decode(self.car[index].now_pos), False]) # 시작지점 지도에 찍기

In [224]:
global store_path
store_path = [[] for _ in range(car_number)]
global store_cus
store_cus = [{} for _ in range(car_number)]
global store_capacity
store_capacity = [{key: [] for key in range(60)} for _ in range(car_number)]
global store_des
store_des = [[] for _ in range(car_number)]

In [225]:
tra = Simulator()
tra.reset()

In [226]:
tra.run(3600)

custmoer generation 0 17 0
0 th car go to ride new customer []
1 th car go to ride new customer []
2 th car go to ride new customer []
3 th car go to ride new customer []
now_pos 0 [3, 3, 5, 5]
test [[], [], [], []]
station [0, 0, 0, 0, 0, 0, 0, 0]
0 th car path 4
1 th car path 2
2 th car path 6
3 th car path 4
decision 0 

0 th car moving 3
car_time wy7vpfsc wy7uyxm0
0 th car moving 3 840
1 th car moving 3
car_time wy7vpfsc wy7uzufk
1 th car moving 3 600
2 th car moving 5
car_time wy7uygct wy7uzkb4
2 th car moving 5 300
3 th car moving 5
car_time wy7uygct wy7uyxm0
3 th car moving 5 360
custmoer generation 60 24 60
0 th car go to ride new customer [(<__main__.Customer object at 0x0000015D2F14B510>, 6)]
1 th car go to ride new customer [(<__main__.Customer object at 0x0000015D2EF7E3D0>, 1)]
2 th car go to ride new customer []
3 th car go to ride new customer []
now_pos 60 [3, 3, 5, 5]
test [[], [], [], []]
station [0, 0, 0, 0, 0, 0, 0, 0]
0 th car path 4
1 th car path 2
2 th car path 6


In [241]:
import folium.plugins as plugins

def visual_ith_car_path(cari):
    last = 0
    for i, ele in enumerate(store_path[cari]):
        pos, bi = ele
        if bi == False:
            if i == 0:
                folium.Marker(pos, 
                  popup=f'CarNumber: {cari}', 
                  icon=folium.Icon(color='white', prefix='fa')
                 ).add_to(m)
        else:
            plugins.AntPath(
                locations=[ele[0] for ele in store_path[cari][last:i+1]], dash_array=[20, 30]
            ).add_to(m)
            last = i
    plugins.AntPath(
        locations=[ele[0] for ele in store_path[cari][last:]], dash_array=[20, 30]
    ).add_to(m)

    for i, temp_cus in enumerate(list(store_cus[cari].keys())):
        time_list = store_cus[cari][temp_cus]
        start, end = temp_cus.up_pos, temp_cus.down_pos
        folium.Marker(geohash.decode(start), 
          popup=f'WaitingTime: {(time_list[0]-temp_cus.time)//60}min', 
          icon=folium.Icon(icon = str(i+1), color='red', prefix='fa')
         ).add_to(m)
        if len(time_list) > 1:
            folium.Marker(geohash.decode(end), 
              popup=f'BoardingTime: {(time_list[1]-time_list[0])//60}min', 
              icon=folium.Icon(icon = str(i+1), color='blue', prefix='fa')
             ).add_to(m)
            
    for i, station_pos in enumerate(station):
        if station_pos != tra.initial_pos[cari]:
            if station_inf[i] == True:
                folium.Marker(geohash.decode(station_pos), 
                  popup=f'StationNum: {(i)}th', 
                  icon=folium.Icon(color='black', prefix='fa')
                 ).add_to(m)
            else:
                folium.Marker(geohash.decode(station_pos), 
                  popup=f'StationNum: {(i)}th', 
                  icon=folium.Icon(color='gray', prefix='fa')
                 ).add_to(m)

car_index = 3
m = folium.Map(
location=geohash.decode(tra.initial_pos[car_index]), 
zoom_start=14, tiles='cartodbpositron', width=800, height=500)
visual_ith_car_path(car_index) # 여기서 차량 번호를 바꾼다
m.fit_bounds(m.get_bounds())
m.save("car3route.html")
m

In [ ]:
# 시각화 리스트
# 동승으로 인한 도착지연시간(완)
# 정류장 별 탑승/하차 수요자 수(완)
# 평균 탑승/대기 시간(완)
# 평균 분별 탑승자 수(완)
# 총 탑승자 수/총 운행 수(완)
# 평균 동승자 기대값(완)
# 평균 정차 간 이동시간(완)
# 차량별 이동 거리(완)

In [209]:
# 예비 정류장으로 인해 늘어난 시간
# 0: 5->6 6->7과 5->7 비교
# 1: 2->1 1->0과 2->0 비교 7->6 6->5과 7->5 비교
# 3: 2->1 1->0 과 2->0 비교
a56 = cal_total_time(station[5], station[6])
a67 = cal_total_time(station[6], station[7])
a57 = cal_total_time(station[5], station[7])
a21 = cal_total_time(station[2], station[1])
a10 = cal_total_time(station[1], station[0])
a20 = cal_total_time(station[2], station[0])
a76 = cal_total_time(station[7], station[6])
a65 = cal_total_time(station[6], station[5])
a75 = cal_total_time(station[7], station[5])
print((a56 + a67 - a57 + a21 + a10 - a20 + a76 + a65 - a75 + a21 + a10 - a20)//240)

car_time wy7uygct wy7uzkb4
car_time wy7uzkb4 wy7uxsv8
car_time wy7uygct wy7uxsv8
car_time wy7uzufk wy7uzegr
car_time wy7uzegr wykh8hcp
car_time wy7uzufk wykh8hcp
car_time wy7uxsv8 wy7uzkb4
car_time wy7uzkb4 wy7uygct
car_time wy7uxsv8 wy7uygct
8


In [185]:
# 동승으로 인한 탑승 지연 시간 및 시간 배율
# store_cus만 사용
boarding_time_compare = []
# store_cus
# 탑승한 승객의 승차 구역을 각각 카운트
for cus_per_car in store_cus:
    for tcus in cus_per_car:
        if len(cus_per_car[tcus]) == 2:
            a = cal_total_time(tcus.up_pos, tcus.down_pos)
            b = cus_per_car[tcus][1] - cus_per_car[tcus][0]
            boarding_time_compare.append((b, a))
print(boarding_time_compare)
average = []
diff = []
for ele in boarding_time_compare:
    average.append(ele[0]/ele[1])
    diff.append(ele[0] - ele[1])
print(average)
print(diff)
print("시간 지연 배율", int(sum(average)/len(average) * 100), "%")
print("평균 지연 시간", sum(diff)/len(diff)//60, "min")

car_time wy7vpc94 wy7uzns9
car_time wykj00bk wy7uz5w2
car_time wy7vpc94 wy7uznhq
car_time wy7vpc8x wy7uznq9
car_time wy7vpc8x wy7uz7g6
car_time wy7vpc8b wy7uz754
car_time wy7vpc94 wy7uznhq
car_time wy7vpc91 wy7uz6ch
car_time wy7vpc94 wy7uznkf
car_time wy7vpcej wy7uz6c3
car_time wy7vpcej wy7uz7kj
car_time wy7vpcej wy7uz7vd
car_time wy7vpc91 wy7uz7kv
car_time wy7vpcej wy7uzs1q
car_time wy7vpc91 wy7uze22
car_time wykh8jge wy7uznyk
car_time wykh8nhh wykj052e
car_time wykh8js6 wykj052g
car_time wykhbjc6 wy7vpbcb
car_time wykhbjc6 wy7vpbcb
car_time wy7vpc91 wy7uzs54
car_time wy7vpc91 wy7uzkpf
car_time wy7vpcej wy7uz7kj
car_time wy7vpc8b wy7uze9n
car_time wy7vpc8b wy7uzebx
car_time wy7vpcej wy7uz7mm
car_time wy7vpcej wy7uz7s6
car_time wy7vpcej wy7uz7z9
car_time wy7vpc8b wy7uz7rn
car_time wy7vpc91 wy7uze22
car_time wy7vpcej wy7uz77z
car_time wykhbjc1 wy7uzskm
[(720, 480), (720, 780), (720, 780), (720, 540), (720, 780), (720, 780), (720, 780), (720, 780), (720, 840), (720, 720), (300, 780), (30

In [189]:
# 구역 별 탑승 거절 비율, 0프로로 만들려면 얼마나 차량이 있어야 할지 판단에 도움을 줄 수 있을 듯
# store_cus만 사용
# total_boarded_cus_per_area = [0 for _ in range(len(area_inf))]
# # store_cus
# # 탑승한 승객의 승차 구역을 각각 카운트
# for cus_per_car in store_cus:
#     for tcus in cus_per_car:
#         total_boarded_cus_per_area[cal_area_num(hashtoarrayindex(tcus.up_pos))] += 1
# print("구역별 탑승자 수", total_boarded_cus_per_area)

total_up_cus_per_station = [0 for _ in range(len(station_inf))]
total_down_cus_per_station = [0 for _ in range(len(station_inf))]
for i in range(60):
    for tcus in time_lambda[i]:
        thash1 = tcus[0]
        thash2 = tcus[1]
        total_up_cus_per_station[tra.nearest_station(hashtoarrayindex(thash1))] += 1
        total_down_cus_per_station[tra.nearest_station(hashtoarrayindex(thash2))] += 1
print("정류장별 승차 수요자 수", total_up_cus_per_station)
print("정류장별 하차 수요자 수", total_down_cus_per_station)
# 각 구역에서 호출한 승객 수

정류장별 승차 수요자 수 [51, 0, 5, 237, 0, 8, 2, 884]
정류장별 하차 수요자 수 [1, 100, 24, 213, 28, 9, 102, 710]


In [190]:
# 평균 대기 시간, 평균 탑승 시간
waiting_time_dis = []
boarding_time_dis = []
for cus_per_car in store_cus:
    for tcus in cus_per_car:
        ele = cus_per_car[tcus]
        waiting_time_dis.append(ele[0] - 1 - tcus.time)
        if len(ele) > 1:
            boarding_time_dis.append(ele[1] - ele[0])
print(waiting_time_dis)
print(boarding_time_dis)
print("mean_waiting_time", sum(waiting_time_dis)/len(waiting_time_dis)//60, "min")
print("mean_boarding_time", sum(boarding_time_dis)/len(boarding_time_dis)//60, "min")

[1620, 1500, 1500, 1500, 1320, 1260, 1200, 1080, 1020, 1020, 840, 1140, 1080, 780, 660, 660, 240, 1860, 1440, 1320, 1500, 1320, 1920, 1680, 1680, 1500, 1380, 1380, 1200, 1200, 1080, 960, 420, 360, 1920, 1860, 1800, 1740, 1440, 1440, 1380, 1320, 1320, 1200, 1140, 1080, 600]
[720, 720, 720, 720, 720, 720, 720, 720, 720, 720, 300, 300, 300, 300, 300, 1020, 660, 660, 660, 660, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300]
mean_waiting_time 20.0 min
mean_boarding_time 8.0 min


In [192]:
# 평균 분별 탑승자수
cus_per_min = [[0 for _ in range(60)] for _ in range(car_number)]
for j, cus_per_car in enumerate(store_capacity):
    for i in range(60):
        cus_per_min[j][i] += len(cus_per_car[i])
print(cus_per_min)
mean_capacity = []
for i in range(car_number):
    mean_capacity.append(round(sum(cus_per_min[i])/60))
print("평균 분별 탑승자 수", mean_capacity)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 15, 15, 15, 15, 15, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]]
평균 분별 탑승자 수 [3, 1, 2, 3]


In [193]:
# 총 탑승자 수, 운행 완료 수
total_boarding_cus = []
total_complete_cus = [0 for _ in range(car_number)]
for i, cus_per_car in enumerate(store_cus):
    total_boarding_cus.append(len(cus_per_car))
    for cus in cus_per_car:
        ele = cus_per_car[cus]
        if len(ele) > 1:
            total_complete_cus[i] += 1
print("차량별 탑승자 수", total_boarding_cus)
print("차량별 운행완료 수", total_complete_cus)

차량별 탑승자 수 [11, 7, 14, 15]
차량별 운행완료 수 [10, 6, 4, 12]


In [194]:
already_cus_num = [[] for _ in range(car_number)]
for i, cus_per_car in enumerate(store_cus):
    for cus in cus_per_car:
        ele = cus_per_car[cus]
        minu = (ele[0]-1)//60
        already_cus_num[i].append(len(store_capacity[i][minu])) 
print(already_cus_num)
already_cus_num = [round(sum(percar)/len(percar)) for percar in already_cus_num]
print("차량별 탔을때 평균 동승자 수", already_cus_num)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 5, 1], [0, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4], [0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]]
차량별 탔을때 평균 동승자 수 [0, 1, 3, 2]


In [195]:
between_time = [[] for _ in range(car_number)]
moving_dis = [0 for _ in range(car_number)]
for i in range(car_number):
    for j in range(len(store_des[i]) - 1):
        between_time[i].append(cal_total_time(store_des[i][j], store_des[i][j+1]))
        moving_dis[i] += cal_distance(store_des[i][j], store_des[i][j+1])
average_bt = [int(sum(ele)/len(ele)) for ele in between_time]
print(between_time)
print("차량별 다음 정차까지 평균 시간", average_bt, "seconds")
moving_dis = [ele//1000 for ele in moving_dis]
print("차량별 이동 거리", moving_dis, "km")

car_time wy7uyxm0 wy7uyxm0
car_time wy7uyxm0 wy7uyxm0
car_time wy7uyxm0 wy7uyxm0
car_time wy7uyxm0 wy7uyxm0
car_time wy7uyxm0 wy7uyxm0
car_time wy7uyxm0 wy7uyxm0
car_time wy7uyxm0 wy7uyxm0
car_time wy7uyxm0 wy7uyxm0
car_time wy7uyxm0 wy7uyxm0
car_time wy7uyxm0 wy7uyxm0
car_time wy7uyxm0 wy7uyxm0
car_time wy7uyxm0 wy7uyxm0
car_time wy7uyxm0 wy7uyxm0
car_time wy7uyxm0 wy7uyxm0
car_time wy7uyxm0 wy7uygct
car_time wy7uygct wy7uygct
car_time wy7uygct wy7uygct
car_time wy7uygct wy7uygct
car_time wy7uygct wy7uygct
car_time wy7uygct wy7uygct
car_time wy7uygct wy7uygct
car_time wy7uygct wy7uygct
car_time wy7uygct wy7uygct
car_time wy7uygct wy7uygct
car_time wy7uygct wy7uygct
car_time wy7uygct wy7uygct
car_time wy7uygct wy7uygct
car_time wy7uygct wy7uygct
car_time wy7uygct wy7uygct
car_time wy7uygct wy7uzkb4
car_time wy7uzkb4 wy7uzkb4
car_time wy7uzkb4 wy7uzkb4
car_time wy7uzkb4 wy7uzkb4
car_time wy7uzkb4 wy7uzkb4
car_time wy7uzkb4 wy7uzkb4
car_time wy7uzkb4 wy7uxsv8
car_time wy7uxsv8 wy7uxsv8
c

In [198]:
# 정류장별 승차, 하차자 수
up_cus_per_station = [0 for _ in range(len(station_inf))]
down_cus_per_station = [0 for _ in range(len(station_inf))]

for i, store_cus_car in enumerate(store_cus):
    for cus in store_cus_car:
        thash1, thash2 = cus.up_pos, cus.down_pos
        up_cus_per_station[tra.nearest_station(hashtoarrayindex(thash1))] += 1
        if len(store_cus_car[cus]) > 1:
            down_cus_per_station[tra.nearest_station(hashtoarrayindex(thash2))] += 1
            
print("정류장별 승차자 수", up_cus_per_station)
print("정류장별 하차자 수", down_cus_per_station)

num = []
for i, ele in enumerate(total_up_cus_per_station):
    if ele != 0:
        num.append(up_cus_per_station[i]/ele)
print(num)
print("추천차량 대수", int(1/(sum(num)/len(num))))

정류장별 승차자 수 [3, 0, 4, 38, 0, 2, 0, 0]
정류장별 하차자 수 [0, 17, 0, 4, 0, 0, 11, 0]
[0.058823529411764705, 0.8, 0.16033755274261605, 0.25, 0.0, 0.0]
추천차량 대수 4
